# Speedup Training Using GPUs

In this tutorial, you will learn:

* How to copy graph and feature data to GPU.
* Train a GNN model on GPU.

In [ ]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools

## Copy graph and feature data to GPU

We first load the Zachery's Karate club graph and node labels as from the previous sessions.

In [ ]:
from tutorial_utils import load_zachery

# ----------- 0. load graph -------------- #
g = load_zachery()
print(g)

Right now the graph and all its feature data are stored in CPU. Use the `to` API to copy them to another device.

In [ ]:
print('Current device:', g.device)
g = g.to('cuda:0')
print('New device:', g.device)

Verify that features are also copied to GPU.

In [ ]:
print(g.ndata['club'].device)
print(g.ndata['club_onehot'].device)

## Create a GNN model on GPU

The step is the same as creating a CNN or RNN model on GPU. In PyTorch, one can use the `to` API to achieve so.

In [ ]:
# ----------- 1. node features -------------- #
node_embed = nn.Embedding(g.number_of_nodes(), 5)  # Every node has an embedding of size 5.
# Copy node embeddings to GPU
node_embed = node_embed.to('cuda:0')
inputs = node_embed.weight                         # Use the embedding weight as the node features.
nn.init.xavier_uniform_(inputs)

The community label is stored in the `'club'` node feature (0 for instructor, 1 for club president). Only nodes 0 and 33 are labeled.

In [ ]:
labels = g.ndata['club']
labeled_nodes = [0, 33]
print('Labels', labels[labeled_nodes])

### Define a GraphSAGE model

Our model consists of two layers, each computes new node representations by aggregating neighbor information. The equations are:

$$
h_{\mathcal{N}(v)}^k\leftarrow \text{AGGREGATE}_k\{h_u^{k-1},\forall u\in\mathcal{N}(v)\}
$$

$$
h_v^k\leftarrow \sigma\left(W^k\cdot \text{CONCAT}(h_v^{k-1}, h_{\mathcal{N}(v)}^k) \right)
$$

DGL provides implementation of many popular neighbor aggregation modules. They all can be invoked easily with one line of codes. See the full list of supported [graph convolution modules](https://docs.dgl.ai/api/python/nn.pytorch.html#module-dgl.nn.pytorch.conv).

In [ ]:
from dgl.nn import SAGEConv

# ----------- 2. create model -------------- #
# build a two-layer GraphSAGE model
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'mean')
        self.conv2 = SAGEConv(h_feats, num_classes, 'mean')
    
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h
    
# Create the model with given dimensions 
# input layer dimension: 5, node embeddings
# hidden layer dimension: 16
# output layer dimension: 2, the two classes, 0 and 1
net = GraphSAGE(5, 16, 2)

Copy the network to GPU

In [ ]:
net = net.to('cuda:0')

In [ ]:
# ----------- 3. set up loss and optimizer -------------- #
# in this case, loss will in training loop
optimizer = torch.optim.Adam(itertools.chain(net.parameters(), node_embed.parameters()), lr=0.01)

# ----------- 4. training -------------------------------- #
all_logits = []
for e in range(100):
    # forward
    logits = net(g, inputs)
    
    # compute loss
    logp = F.log_softmax(logits, 1)
    loss = F.nll_loss(logp[labeled_nodes], labels[labeled_nodes])
    
    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    all_logits.append(logits.detach())
    
    if e % 5 == 0:
        print('In epoch {}, loss: {}'.format(e, loss))

In [ ]:
# ----------- 5. check results ------------------------ #
pred = torch.argmax(logits, axis=1)
print('Accuracy', (pred == labels).sum().item() / len(pred))

**What if the graph and its feature data cannot fit into one GPU memory?**

* Instead of running a GNN on the full graph, run it on some sample subgraphs till converge.
* Issue different samples to different GPUs to enjoy even more acceleration.
* Partition the graph to multiple machines and train it distributedly.

Our later sessions will cover each of these methods.